In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import scipy.special as spe
from hansolo import *
import psutil
from sklearn import datasets
from sklearn.model_selection import train_test_split

#retrieve dataset
digits = datasets.load_digits()
images=digits.images/16 #(1797,8,8)
target=digits.target #(1797,)

X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.2, random_state=42)
M_train = np.shape(X_train)[0]
M_test = np.shape(X_test)[0]

# Reshape images and compute probabilities
flattened_images_train = X_train.reshape((M_train, -1))
flattened_images_test = X_test.reshape((M_test, -1))

eta_mid = 3 #parameter EWA 1st hidden layer
expert1 = 'EWA' #expert 1st hidden layer

eta_mid2 = 3 #parameter EWA 2nd hidden layer
expert2 = 'EWA' #expert 2nd hidden layer

eta_output = 0.03 #parameter EWA output layer
expert3 = 'EWA' #expert output layer

M0 = 40 #number of images to train 1st hidden layer
M0_sel = 40 #number of images to select neurons 1st hidden layer
M1 = 40 #number of images to train 2nd hidden layer
M1_sel = 40 #number of images to select hidden neurons 2nd layer
M_mid = M0 + M0_sel + M1 + M1_sel
M_out = M_train - M_mid #number of images to train output neurons
M_tot = M_mid + M_out + M_test

N = 2000 #number of time steps
p = 1  # spiking proba input neurons 
nu = 0.5 # bias

K_input = 64 #number of input neurons

K_mid_new = 100 #number of selected neurons 1st hidden layer
K_mid2_new = 150 #number of selected neurons 2nd hidden layer

K_mid = int(K_input * (K_input - 1)/2) #number of neurons 1st hidden layer before selection
K_output = 10  #number of output neurons

images_mid = flattened_images_train[:M_mid,:] /16 
images_out = flattened_images_train[M_mid:M_mid+M_out,:] /16
images_test = flattened_images_test /16

target_out = y_train[M_mid:]
target_test = y_test

def same_corr(a,b):
    return a[0] == b[0] or a[0] == b[1] or a[1] == b[0] or a[1] == b[1]
        
indices = np.arange(K_input)
i, j = np.meshgrid(indices, indices)
mask = i < j
l_mid = np.column_stack((i[mask], j[mask])) #tensor indices hidden neurons 1st layer

Nb = 10 #number of simulations
accuracy = np.zeros((Nb))
for nb in range(Nb):
    #simulation input neurons

    prob =  np.transpose(np.array([p*images_mid for i in range(N)]), axes = (2,0,1))

    # Input Neurons activity (K_input,N,M)

    input_neurons_mid = stats.bernoulli.rvs(prob)
    prob_input = np.transpose(np.array([p*images_out for i in range(N)]), axes = (2,0,1))

    input_neurons_out = stats.bernoulli.rvs(prob_input)

    prob_input = np.transpose(np.array([p*images_test for i in range(N)]), axes = (2,0,1))
    input_neurons_test = stats.bernoulli.rvs(prob_input)

    #M0 = 30 #number of images for training of mid neurons

    cum_gain_mid = np.zeros((K_mid))
    cum_gain_input_to_mid = np.zeros((K_mid,K_input))

    #training mid neurons

    cum_gain_EWA = np.zeros((K_mid, K_input))

    for m in range(M0-1):
        gain = GainMid(np.arange(K_mid),K_mid,K_input,input_neurons_mid[:,:,m], l_mid)
        cum_gain_EWA += gain
        #print(m)
    if expert1 == 'EWA':
        W_mid_end = spe.softmax(eta_mid * cum_gain_EWA, axis = 1)
    

    #selection mid 

    prob_mid = -nu + np.einsum('ai, itm -> atm', W_mid_end, input_neurons_mid[:, :, M0:M0 + M0_sel])
    clipped_probas_mid = np.clip(prob_mid, 0, 1)
    mid_neurons_sel = stats.bernoulli.rvs(clipped_probas_mid)
    P_mid = np.sum(mid_neurons_sel, axis=1) / N
    max_P = np.max(P_mid, axis = 1 )
    max_P_sorted = np.sort(max_P)

    x = max_P_sorted[K_mid-K_mid_new] 
    mid_neur_ok = np.where(max_P > x)[0] #selected neurons
    missing = K_mid_new - np.shape(mid_neur_ok)[0]
    mid_neur_to_sel = np.where(max_P == x)[0]
    n_tot = np.shape(mid_neur_to_sel)[0]
    selected_indices = np.random.choice(n_tot, size=missing, replace=False) #random choice of neurons which have the same nb of spikes
    mid_neur_missing = mid_neur_to_sel[selected_indices]

    mid_neur = np.concatenate((mid_neur_missing, mid_neur_ok))
    mid_neur = np.sort(mid_neur)

    #simu mid neurons
    prob_mid = -nu + np.einsum('ai, itm -> atm', W_mid_end[mid_neur,:], input_neurons_mid[ :, :, M0+M0_sel:])
    clipped_probas_mid = np.clip(prob_mid, 0, 1)
    mid_neurons_mid2=stats.bernoulli.rvs(clipped_probas_mid)

    #mid2
    l_mid2 = []
    for i in range(K_mid_new):
        for j in range(K_mid_new):
            if i < j and not same_corr(l_mid[mid_neur[i]], l_mid[mid_neur[j]]):
                l_mid2.append([i,j])
    l_mid2 = np.array(l_mid2)
    K_mid2 = np.shape(l_mid2)[0]

    cum_gain_mid_to_mid2 = np.zeros((K_mid2,K_mid_new))

    for m in range(M1-1):
        gain_mid2 = GainMid(np.arange(K_mid2),K_mid2,K_mid_new,mid_neurons_mid2[:,:,m], l_mid2) #(K_mid,K_input) a vectoriser !
        cum_gain_mid_to_mid2 += gain_mid2
        #print(m)
    
    if expert2 == "EWA":
        W_mid2_end = spe.softmax(eta_mid2 * cum_gain_mid_to_mid2, axis = 1)

    #selection mid2
        
    prob_mid2 = -nu + np.einsum('ai, itm -> atm', W_mid2_end, mid_neurons_mid2[:, :, M1:M1+M1_sel])
    clipped_probas_mid2 = np.clip(prob_mid2, 0, 1)
    mid2_neurons_sel= stats.bernoulli.rvs(clipped_probas_mid2)
    P_mid2 = np.sum(mid2_neurons_sel, axis=1) / N
    max_P2 = np.max(P_mid2, axis = 1 )
    max_P2_sorted = np.sort(max_P2)

    x = max_P2_sorted[K_mid2-K_mid2_new] 
    mid2_neur_ok = np.where(max_P2 > x)[0] #selected neurons
    missing2 = K_mid2_new - np.shape(mid2_neur_ok)[0]
    mid2_neur_to_sel = np.where(max_P2 == x)[0]
    n_tot2 = np.shape(mid2_neur_to_sel)[0]
    selected_indices = np.random.choice(n_tot2, size=missing2, replace=False) #random choice of neurons which have the same nb of spikes
    mid2_neur_missing = mid2_neur_to_sel[selected_indices]

    mid2_neur = np.concatenate((mid2_neur_missing, mid2_neur_ok))
    mid2_neur = np.sort(mid2_neur)

    prob_mid = -nu + np.einsum('ai, itm -> atm', W_mid_end[mid_neur,:], input_neurons_out)
    clipped_probas_mid = np.clip(prob_mid, 0, 1)
    mid_neurons_out=stats.bernoulli.rvs(clipped_probas_mid)

    prob_mid2 = -nu + np.einsum('ai, itm -> atm', W_mid2_end[mid2_neur,:], mid_neurons_out)
    clipped_probas_mid2 = np.clip(prob_mid2, 0, 1)
    mid2_neurons_out=stats.bernoulli.rvs(clipped_probas_mid2)

        
    P_mid2 = np.sum(mid2_neurons_out, axis = 1) / N

    #simu test
    prob_mid = -nu + np.einsum('ai, itm -> atm', W_mid_end[mid_neur,:], input_neurons_test)
    clipped_probas_mid = np.clip(prob_mid, 0, 1)
    mid_neurons_test=stats.bernoulli.rvs(clipped_probas_mid)

    prob_mid2 = -nu + np.einsum('ai, itm -> atm', W_mid2_end[mid2_neur,:], mid_neurons_test)
    clipped_probas_mid2 = np.clip(prob_mid2, 0, 1)
    mid2_neurons_test=stats.bernoulli.rvs(clipped_probas_mid2)

    cum_gain_mid2_to_output = np.zeros((K_output, K_mid2_new))

    for m in range(M_out-1):
        gain_output = GainOutput(K_output, K_mid2_new, P_mid2[:,m], target_out[m]) 
        cum_gain_mid2_to_output += gain_output
        
    if expert3 == "EWA":
        W_output_end = spe.softmax(eta_output * cum_gain_mid2_to_output, axis =1 )

    #test new images

    answers_test = np.zeros((M_test))

    for m in range(M_test):
        
        # Simulation output neurons (without Kalikow)
        probas_output = np.sum(W_output_end[:,None,:] * mid2_neurons_test[:,:,m].T, axis = 2)
        probas_output = np.clip(probas_output, 0, 1)
        output_neurons = stats.bernoulli.rvs(probas_output)

        P_output = np.sum(output_neurons, axis = 1) / N
        chosen_class = np.where(P_output == np.max(P_output))[0]

        if chosen_class[0] == target_test[m]:
            answers_test[m] = 1
    accuracy[nb] = np.sum(answers_test)/M_test
    print(nb, accuracy[nb])

#np.save(f"accuracy2mid_EWA_{K_mid2_new}_{eta_output}_10", accuracy)

